In [1]:
from bs4 import BeautifulSoup
import re
import urllib2
from matplotlib import pyplot as plt
import sqlite3 as sql
from selenium import webdriver
from selenium.webdriver.common.keys import Keys

In [2]:
#Problem 1

BANKS = ['JPMORGAN', 'CAPITAL ONE', 'DISCOVER']

bank_data = BeautifulSoup(open('Big Bank Info.htm'))

data = list()

for bank in BANKS:
    info = bank_data.find(text=re.compile(bank))
    info = info.parent.parent.find_all('td')
    info = [str(info[0].string), str(info[1].string), str(info[2].string), str(info[6].string), str(info[9].string)]
    data.append(info)



C:\Users\Joey\Anaconda\lib\site-packages\bs4\__init__.py:166: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

To get rid of this warning, change this:

 BeautifulSoup([your markup])

to this:

 BeautifulSoup([your markup], "lxml")

  markup_type=markup_type))


In [2]:
#Problem 2

url = 'http://www.wunderground.com/history/airport/KSAN/2015/1/1/DailyHistory.html?req_city=San+Diego&req_state=CA&req_statename=California&reqdb.zip=92101&reqdb.magic=1&reqdb.wmo=99999&MR=1'

url_content = urllib2.urlopen(url).read()
soup = BeautifulSoup(url_content)
max_temp = soup.find(text=re.compile('Max Temperature'))
print(max_temp.parent.parent.next_sibling.next_sibling.find(class_='wx-value').text)

next_tag = soup.find(text=re.compile('Next Day'))
print(next_tag.parent)
print(next_tag.parent['href'])

59
<a href="/history/airport/KSAN/2015/1/2/DailyHistory.html">Next Day »</a>
/history/airport/KSAN/2015/1/2/DailyHistory.html


C:\Users\Joey\Anaconda\lib\site-packages\bs4\__init__.py:166: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

To get rid of this warning, change this:

 BeautifulSoup([your markup])

to this:

 BeautifulSoup([your markup], "lxml")

  markup_type=markup_type))


In [4]:
#Problem 3

weather_url = 'https://www.wunderground.com/history/airport/KSAN/2014/1/1/DailyHistory.html'
weather_content = urllib2.urlopen(weather_url).read()
weather_soup = BeautifulSoup(weather_content)
actual = []
while('2015' not in weather_soup.find(class_='history-date').string):
    while(len(weather_soup.find_all(string='Actual')) != 1):
        weather_content = urllib2.urlopen(weather_url).read()
        weather_soup = BeautifulSoup(weather_content)
    actual_temp = weather_soup.find(string='Max Temperature').parent.parent.next_sibling.next_sibling.next_sibling.next_sibling.span.span.text
    actual.append(int(actual_temp))
    next_url = weather_soup.find(string=re.compile('Next Day')).parent['href']
    weather_url = 'https://www.wunderground.com'+next_url
    weather_content = urllib2.urlopen(weather_url).read()
    weather_soup = BeautifulSoup(weather_content)
    
X = xrange(365)
Y = actual
plt.plot(X,Y)
plt.show()
plt.close()

In [5]:
#Problem 4, option 1

db = sql.connect('soup')
cur = db.cursor()
cur.execute('drop table if exists Sectors')
cur.execute('CREATE TABLE Sectors (Name TEXT, Abbreviation TEXT, Percent TEXT, Cap TEXT);')

finance_url = 'http://www.google.com/finance'
finance_content = urllib2.urlopen(finance_url).read()
finance_soup = BeautifulSoup(finance_content)

#print finance_soup
Sectors = finance_soup.find(string=re.compile('Sector summary')).parent.parent.parent.next_sibling.next_sibling.find_all(href=True)


i=0

for sector in Sectors:
    i+=1
    
    sector_url = str('http://www.google.com'+sector['href'])
    sector_content = urllib2.urlopen(sector_url).read()
    sector_soup = BeautifulSoup(sector_content)
        
        
        
    Gainers_tag = sector_soup.find(string=re.compile('Gainers')).parent.parent.parent.next_sibling

    Name = Gainers_tag.find_all(href=True)[0].text
    Abbrev = Gainers_tag.find_all(href=True)[1].text
    Percent = Gainers_tag.find_all('span')[1].text
    Cap = Gainers_tag.find_all('td')[4].text
    
    cur.execute('INSERT INTO Sectors VALUES (?,?,?,?)',[Name, Abbrev, Percent, Cap])    

db.close()

In [14]:
#Problem 4, option 2

In [16]:
#Problem 4, option 3

In [3]:
#Problem 5

driver = webdriver.Firefox()
NBA_url = 'http://stats.nba.com/league/team/#!/?sort=W&dir=1'
driver.get(NBA_url)

NBA_soup = BeautifulSoup(driver.page_source)
name_tag = NBA_soup.find_all('td', class_='team-name first')
Name = [i.a for i in name_tag]

driver.quit()

WebDriverException: Message: The browser appears to have exited before we could connect. If you specified a log_file in the FirefoxBinary constructor, check it for details.


In [7]:
#Problem 6

db = sql.connect('NBA')
cur = db.cursor()
cur.execute('drop table if exists Stats')
cur.execute('create table Stats (Name TEXT, Wins TEXT, Losses TEXT)')

for i in Name:
    #print i.text
    name = i.text
    wins = i.parent.parent.find_all('td')[2].text
    losses = i.parent.parent.find_all('td')[3].text
    
    cur.execute('insert into Stats values (?,?,?)',[name, wins, losses])

db.close()